In [ ]:
import pandas as pd

df = pd.read_excel("/Users/soo/repos/private/uci_online_retail/data/online_retail_dataset.xlsx")


In [ ]:

print(type(df["InvoiceDate"]))
print(df["InvoiceDate"].dtype)


In [ ]:
import sqlite3
conn = sqlite3.connect("/Users/soo/repos/private/uci_online_retail/data/retail.db")
cursor = conn.cursor()
cursor.execute("SELECT julianday('2011-12-05 10:00:00') - julianday('2011-12-01 08:00:00')")
result = cursor.fetchone()[0]
print(f"Days difference: {result}")  # Should be ~4.08 days